# Автокодировщики

На этом занятии будем обучать глубокие автокодировщики на примере изображений человеческих лиц [lfw dataset](http://vis-www.cs.umass.edu/lfw/).

# Начинаем работу

In [ ]:
import tensorflow as tf
import keras, keras.layers as L, keras.backend as K
import numpy as np
from sklearn.model_selection import train_test_split
from lfw_dataset import load_lfw_dataset
%matplotlib inline
import matplotlib.pyplot as plt
import download_utils
import keras_utils
import numpy as np
from keras_utils import reset_tf_session

# Загружаем данные
Оригинальные ссылки:
- http://www.cs.columbia.edu/CAVE/databases/pubfig/download/lfw_attributes.txt
- http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz
- http://vis-www.cs.umass.edu/lfw/lfw.tgz

In [ ]:
# load images
X, attr = load_lfw_dataset(use_raw=True, dimx=32, dimy=32)
IMG_SHAPE = X.shape[1:]

# center images
X = X.astype('float32') / 255.0 - 0.5

# split
X_train, X_test = train_test_split(X, test_size=0.1, random_state=42)

In [ ]:
def show_image(x):
    plt.imshow(np.clip(x + 0.5, 0, 1))

In [ ]:
plt.title('sample images')

for i in range(6):
    plt.subplot(2,3,i+1)
    show_image(X[i])

print("X shape:", X.shape)
print("attr shape:", attr.shape)

# try to free memory
del X
import gc
gc.collect()

# Архитектура автокодировщика

Будем строить архитектуру сети как две последовательные модели в Keras: кодировщик и декодировщик. Затем применяем обычное API для обучения моделей.

<img src="autoencoder.png" style="width:50%">

# Первый шаг: метод главных компонент

Метод главных компонент (PCA) - популярный метод сокращения размерности.


PCA пытается декомопзировать матрицу объекты-признаки на две матрице меньшего размера $W$ и $\hat W$, минимизируя средний квадрат ошибки:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - объектная матрица (**центрированная**);
- $W \in \mathbb{R}^{m \times d}$ - матрица прямой трансформации;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - матрица обратной трансформации;
- $n$ примеров, $m$ - начальная размерность, $d$ - итоговая размерность;

Геометрическая интерпретация: мы хотим найти $d$ осей по которым наблюдается наибольшая дисперсия.

<img src="pca.png" style="width:30%">


PCA можно рассмаривать как особый случай автокодировщика.

* __Encoder__: X -> Dense(d units) -> код
* __Decoder__: код -> Dense(m units) -> X

Здесь Dense это полносвязаный слой с линейной активацией:   $f(X) = W \cdot X + \vec b $

In [ ]:
def build_pca_autoencoder(img_shape, code_size):
    """
    Here we define a simple linear autoencoder as described above.
    We also flatten and un-flatten data to be compatible with image shapes
    """
    
    encoder = keras.models.Sequential()
    ## ВАШ КОД: добавляем простые слои

    decoder = keras.models.Sequential()
    ## ВАШ КОД: добавляем обратные слои
    
    return encoder,decoder

Объединяем в одну модель:

In [ ]:
s = reset_tf_session()

encoder, decoder = build_pca_autoencoder(IMG_SHAPE, code_size=32)

inp = L.Input(IMG_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = keras.models.Model(inputs=inp, outputs=reconstruction)
autoencoder.compile(optimizer='adamax', loss='mse')

# Обучаем модель! epochs=15, callbaback=[keras_utils.TqdmProgressCallback()], validation, verbose=0

In [ ]:
def visualize(img,encoder,decoder):
    """Draws original, encoded and decoded images"""
    code = encoder.predict(img[None])[0]  # img[None] is the same as img[np.newaxis, :]
    reco = decoder.predict(code[None])[0]

    plt.subplot(1,3,1)
    plt.title("Original")
    show_image(img)

    plt.subplot(1,3,2)
    plt.title("Code")
    plt.imshow(code.reshape([code.shape[-1]//2,-1]))

    plt.subplot(1,3,3)
    plt.title("Reconstructed")
    show_image(reco)
    plt.show()


In [ ]:
score = autoencoder.evaluate(X_test,X_test,verbose=0)
print("PCA MSE:", score)

In [ ]:
img = X_test[2]
visualize(img,encoder,decoder)

# Сверточный автокодировщик

Добавим еще большей слоев в модель.

## Кодировщик

Кодировщик будет довольно стандартный - составляем сверточные слои и взятие максимума для получения представления нужного размера (`code_size`).

Будем использовать `activation='elu'`для всех полносвязных и сверточных слоев.

Начнем с 4 пар (conv, pool) с ядром (3, 3), `padding='same'` и выходными каналами `32, 64, 128, 256`. Плюс не забываем про вытягивание (`L.Flatten()`) вызода перед последним полносвязным слоем!

## Декодировщик
Для декодировщика будем использовать "транспонированную свертку". 

Обычный полносвязный слой по части изображения генерирует число (patch -> number). В "транспонированной свертке" мы производим обратную операцию (number -> patch), производя "разворачивание" сверток кодировщика( см. [this video](https://www.coursera.org/learn/intro-to-deep-learning/lecture/auRqf/a-glimpse-of-other-computer-vision-tasks) с 5:41).

<img src="transpose_conv.jpg" style="width:60%">
Здесь используется сдвиг 2 для генерации выхода размером 4x4, производя и обратную операцию взятия максимума. .

Как это делается в Keras:
```python
L.Conv2DTranspose(filters=?, kernel_size=(3, 3), strides=2, activation='elu', padding='same')
```

Будем строить декодер, начиная с полносвязного слоя (помним про reshape для обратной операции к `L.Flatten()`). Затем мы будем поочередно обращать все пары (conv, pool): составляем 4 `L.Conv2DTranspose` слоя с layers with the following numbers of output channelsобратным порядком выходных каналов: `128, 64, 32, 3`. Для последнего  `L.Conv2DTranspose` слоя возьмем `activation=None`, т.к. это итоговая картинка.

In [ ]:
# Let's play around with transpose convolution on examples first
def test_conv2d_transpose(img_size, filter_size):
    print("Transpose convolution test for img_size={}, filter_size={}:".format(img_size, filter_size))
    
    x = (np.arange(img_size ** 2, dtype=np.float32) + 1).reshape((1, img_size, img_size, 1))
    f = (np.ones(filter_size ** 2, dtype=np.float32)).reshape((filter_size, filter_size, 1, 1))

    s = reset_tf_session()
    
    conv = tf.nn.conv2d_transpose(x, f, 
                                  output_shape=(1, img_size * 2, img_size * 2, 1), 
                                  strides=[1, 2, 2, 1], 
                                  padding='SAME')

    result = s.run(conv)
    print("input:")
    print(x[0, :, :, 0])
    print("filter:")
    print(f[:, :, 0, 0])
    print("output:")
    print(result[0, :, :, 0])
    s.close()
        
test_conv2d_transpose(img_size=2, filter_size=2)
test_conv2d_transpose(img_size=2, filter_size=3)
test_conv2d_transpose(img_size=4, filter_size=2)
test_conv2d_transpose(img_size=4, filter_size=3)

In [ ]:
def build_deep_autoencoder(img_shape, code_size):
    """PCA's deeper brother. See instructions above. Use `code_size` in layer definitions."""
    H,W,C = img_shape
    
    # encoder
    encoder = keras.models.Sequential()
    encoder.add(L.InputLayer(img_shape))
    
    # ВАШ КОД: строим модель

    # decoder
    decoder = keras.models.Sequential()
    decoder.add(L.InputLayer((code_size,)))
    
    encoder_prev_shape = (img_shape[0] // 2**4, img_shape[1] // 2**4, 256)
    # ВАШ КОД: строим модель
    
    return encoder, decoder

In [ ]:
# Check autoencoder shapes along different code_sizes
get_dim = lambda layer: np.prod(layer.output_shape[1:])
for code_size in [1,8,32,128,512]:
    s = reset_tf_session()
    encoder, decoder = build_deep_autoencoder(IMG_SHAPE, code_size=code_size)
    print("Testing code size %i" % code_size)
    assert encoder.output_shape[1:]==(code_size,),"encoder must output a code of required size"
    assert decoder.output_shape[1:]==IMG_SHAPE,   "decoder must output an image of valid shape"
    assert len(encoder.trainable_weights)>=6,     "encoder must contain at least 3 layers"
    assert len(decoder.trainable_weights)>=6,     "decoder must contain at least 3 layers"
    
    for layer in encoder.layers + decoder.layers:
        assert get_dim(layer) >= code_size, "Encoder layer %s is smaller than bottleneck (%i units)"%(layer.name,get_dim(layer))

print("All tests passed!")
s = reset_tf_session()

In [ ]:
# Look at encoder and decoder shapes.
# Total number of trainable parameters of encoder and decoder should be close.
s = reset_tf_session()
encoder, decoder = build_deep_autoencoder(IMG_SHAPE, code_size=32)
encoder.summary()
decoder.summary()

Обучем нашу модель. Займет порядка **1 часа**.

In [ ]:
s = reset_tf_session()

encoder, decoder = build_deep_autoencoder(IMG_SHAPE, code_size=32)

# Создаем модель как в примере с PCA

In [ ]:
# we will save model checkpoints here to continue training in case of kernel death
model_filename = 'autoencoder.{0:03d}.hdf5'
last_finished_epoch = None

#### uncomment below to continue training from model checkpoint
#### fill `last_finished_epoch` with your latest finished epoch
# from keras.models import load_model
# s = reset_tf_session()
# last_finished_epoch = 4
# autoencoder = load_model(model_filename.format(last_finished_epoch))
# encoder = autoencoder.layers[1]
# decoder = autoencoder.layers[2]

In [ ]:
# Обучаем модель! epochs=25, callbaback=[keras_utils.TqdmProgressCallback()], validation, verbose=0

In [ ]:
reconstruction_mse = autoencoder.evaluate(X_test, X_test, verbose=0)
print("Convolutional autoencoder MSE:", reconstruction_mse)
for i in range(5):
    img = X_test[i]
    visualize(img,encoder,decoder)

In [ ]:
# save trained weights
encoder.save_weights("encoder.h5")
decoder.save_weights("decoder.h5")

# Автокодировщик для удаления шума

рассмтоим одно из приложений авктодировщиков - удаление шума.

Как будет работать удаление шума:
<img src="denoising.jpg" style="width:40%">

Будем исопльзовать ту же архитектуру, но измением то, как будем ее обучать: немного повредим наши данные случайнм образом перед каждой эпохой.

Есть много подходов как добавить шум: гауссовский белый шум, перекрытие случайными черными прямоугольниками и т.д. 

In [ ]:
def apply_gaussian_noise(X,sigma=0.1):
    """
    adds noise from standard normal distribution with standard deviation sigma
    :param X: image tensor of shape [batch,height,width,3]
    Returns X + noise.
    """
    noise = np.random.normal(scale=sigma, size=X.shape)
    return X + noise

In [ ]:
# noise tests
theoretical_std = (X_train[:100].std()**2 + 0.5**2)**.5
our_std = apply_gaussian_noise(X_train[:100],sigma=0.5).std()
assert abs(theoretical_std - our_std) < 0.01, "Standard deviation does not match it's required value. Make sure you use sigma as std."
assert abs(apply_gaussian_noise(X_train[:100],sigma=0.5).mean() - X_train[:100].mean()) < 0.01, "Mean has changed. Please add zero-mean noise"

In [ ]:
# test different noise scales
plt.subplot(1,4,1)
show_image(X_train[0])
plt.subplot(1,4,2)
show_image(apply_gaussian_noise(X_train[:1],sigma=0.01)[0])
plt.subplot(1,4,3)
show_image(apply_gaussian_noise(X_train[:1],sigma=0.1)[0])
plt.subplot(1,4,4)
show_image(apply_gaussian_noise(X_train[:1],sigma=0.5)[0])

Обучаем ~ **1 часа**.

In [ ]:
s = reset_tf_session()

# we use bigger code size here for better quality
encoder, decoder = build_deep_autoencoder(IMG_SHAPE, code_size=512)
assert encoder.output_shape[1:]==(512,), "encoder must output a code of required size"

inp = L.Input(IMG_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = keras.models.Model(inp, reconstruction)
autoencoder.compile('adamax', 'mse')

for i in range(25):
    print("Epoch %i/25, Generating corrupted samples..."%(i+1))
    X_train_noise = apply_gaussian_noise(X_train)
    X_test_noise = apply_gaussian_noise(X_test)
    
    # we continue to train our model with new noise-augmented data
    # Обучаем модель! epochs=1, callbaback=[keras_utils.TqdmProgressCallback()], validation, verbose=0

In [ ]:
X_test_noise = apply_gaussian_noise(X_test)
denoising_mse = autoencoder.evaluate(X_test_noise, X_test, verbose=0)
print("Denoising MSE:", denoising_mse)
for i in range(5):
    img = X_test_noise[i]
    visualize(img,encoder,decoder)

# Поиск изображений с помощью автокодировщика

Поддим автокодирощвику изображение и найдем походие изображения в его скрытом пространстве:

<img src="similar_images.jpg" style="width:60%">

Для ускорения процесса извлечения будем использовать локльное хеширование векторов кодирощвика ([technique](https://erikbern.com/2015/07/04/benchmark-of-approximate-nearest-neighbor-libraries.html)), что позволит уменьить число потенциальных соседей нашего изображения. Будем искать ближайших простым перебором.

In [ ]:
# restore trained encoder weights
s = reset_tf_session()
encoder, decoder = build_deep_autoencoder(IMG_SHAPE, code_size=32)
encoder.load_weights("encoder.h5")

In [ ]:
images = X_train
codes = encoder.predict(X_train)
assert len(codes) == len(images)

In [ ]:
from sklearn.neighbors.unsupervised import NearestNeighbors
nei_clf = NearestNeighbors(metric="euclidean")
nei_clf.fit(codes)

In [ ]:
def get_similar(image, n_neighbors=5):
    assert image.ndim==3,"image must be [batch,height,width,3]"

    code = encoder.predict(image[None])
    
    (distances,),(idx,) = nei_clf.kneighbors(code,n_neighbors=n_neighbors)
    
    return distances,images[idx]

In [ ]:
def show_similar(image):
    
    distances,neighbors = get_similar(image,n_neighbors=3)
    
    plt.figure(figsize=[8,7])
    plt.subplot(1,4,1)
    show_image(image)
    plt.title("Original image")
    
    for i in range(3):
        plt.subplot(1,4,i+2)
        show_image(neighbors[i])
        plt.title("Dist=%.3f"%distances[i])
    plt.show()

Несколько примеров:

In [ ]:
# smiles
show_similar(X_test[247])

In [ ]:
# ethnicity
show_similar(X_test[56])

In [ ]:
# glasses
show_similar(X_test[63])

# Быстрый морфинг изобржаений


Можем использовать коды изображений для составления нового путем их линейной комбинации.

In [ ]:
# restore trained encoder weights
s = reset_tf_session()
encoder, decoder = build_deep_autoencoder(IMG_SHAPE, code_size=32)
encoder.load_weights("encoder.h5")
decoder.load_weights("decoder.h5")

In [ ]:
for _ in range(5):
    image1,image2 = X_test[np.random.randint(0,len(X_test),size=2)]

    code1, code2 = encoder.predict(np.stack([image1, image2]))

    plt.figure(figsize=[10,4])
    for i,a in enumerate(np.linspace(0,1,num=7)):

        output_code = code1*(1-a) + code2*(a)
        output_image = decoder.predict(output_code[None])[0]

        plt.subplot(1,7,i+1)
        show_image(output_image)
        plt.title("a=%.2f"%a)
        
    plt.show()